## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_trimmed = application_df.copy()
application_df_trimmed = application_df_trimmed.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
application_df_trimmed.nunique(axis=0)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

## Filter for one application type

In [4]:
application_df_trimmed['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Filter for a single application type and then drop that column
application_df_filtered = application_df_trimmed[application_df_trimmed['APPLICATION_TYPE'] == 'T3']
application_df_filtered = application_df_filtered.drop(['APPLICATION_TYPE'], axis=1)
application_df_filtered.head()

,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
1,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,Independent,C1200,Preservation,Trust,1,0,N,5000,1
6,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1


## Convert categorical data to numerical

In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
# (AFFILIATION,CLASSIFICATION, USE_CASE, ORGANIZATION)
application_df_noncat = pd.get_dummies(data=application_df_filtered, columns=['AFFILIATION', 'CLASSIFICATION', 
                                                                              'USE_CASE', 'ORGANIZATION', 'STATUS',
                                                                              'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])
application_df_noncat.head()

,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C0,CLASSIFICATION_C1000,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
1,108590,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
3,6692,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0
5,5000,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,31452,1,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0


In [7]:
# Split our preprocessed data into our features and target arrays
y = application_df_noncat['IS_SUCCESSFUL']
X = application_df_noncat.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="tanh", input_dim=100))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=12, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                1616      
                                                                 
 dense_1 (Dense)             (None, 12)                204       
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 1833 (7.16 KB)
Trainable params: 1833 (7.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# Compile the model 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# set a callback for every 5 epochs to an HDF5 file
mc = tf.keras.callbacks.ModelCheckpoint(filepath='AlphabetSoupCharity_Optimization1.h5', save_weights_only=True, save_freq=5)

# Train the model over 100 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[mc])

Epoch 1/100
634/634 [==============================] - 3s 4ms/step - loss: 0.5889 - accuracy: 0.7033
Epoch 2/100
634/634 [==============================] - 2s 4ms/step - loss: 0.5692 - accuracy: 0.7192
Epoch 3/100
634/634 [==============================] - 2s 3ms/step - loss: 0.5669 - accuracy: 0.7209
Epoch 4/100
634/634 [==============================] - 2s 4ms/step - loss: 0.5649 - accuracy: 0.7233
Epoch 5/100
634/634 [==============================] - 2s 3ms/step - loss: 0.5634 - accuracy: 0.7242
Epoch 6/100
634/634 [==============================] - 2s 3ms/step - loss: 0.5622 - accuracy: 0.7233
Epoch 7/100
634/634 [==============================] - 2s 3ms/step - loss: 0.5617 - accuracy: 0.7250
Epoch 8/100
634/634 [==============================] - 2s 3ms/step - loss: 0.5603 - accuracy: 0.7253
Epoch 9/100
634/634 [==============================] - 2s 3ms/step - loss: 0.5604 - accuracy: 0.7262
Epoch 10/100
634/634 [==============================] - 2s 3ms/step - loss: 0.5594 - accura

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

212/212 - 0s - loss: 0.5679 - accuracy: 0.7232 - 318ms/epoch - 2ms/step
Loss: 0.5678507685661316, Accuracy: 0.7232248783111572
